In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

header = ['i', 'success', 'method', 'preconditioning', 'dimension', 'point_number',
          'exact', 'h', 'absolute', 'fk', 'gradfk', 'k', 'time']
results_dir = '../results_exact'

method_mapper = {
    0: 'Modified Newton',
    1: 'Truncated Newton',
}

In [4]:
def load_results(func):
    path = os.path.join(results_dir, func, 'results.csv')
    df = pd.read_csv(path, names=header)
    df.method = df.method.apply(lambda x: method_mapper[x])
    df.preconditioning = df.preconditioning.astype(bool)
    df.exact = df.exact.astype(bool)
    df.absolute = df.absolute.astype(bool)
    df.loc[:, 'fname'] = df.i.apply(lambda x: f"experiment_{x}.csv")
    df.set_index('i', inplace=True)
    for i, file in df.fname.items():
        path = os.path.join(results_dir, func, file)
        data = pd.read_csv(path)
        try:
            errors = data['error_norm'].iloc[-3:].values
            rate = np.log(errors[2]/errors[1]) / np.log(errors[1]/errors[0])
        except:
            rate = np.nan
        df.loc[i, 'rate'] = rate
    return df

In [ ]:
def avg_table(df, method, exact, fname, absolute=None):
    table_df = df[
        (df.method == method) &
        (df.exact == exact)
    ]
    # If finite differences, filter by h and add text
    if exact:
        differentiation_text = "with exact gradient and hessian"
        grouping = ['dimension', 'preconditioning']
    else:
        table_df = table_df[df.absolute == absolute]
        differentiation_text = f"with {"absolute" if absolute else "relative"} finite differences"
        grouping = ['dimension', 'h', 'preconditioning']
        table_df.h = table_df.h.apply(lambda x: f"{x:.0e}")
    # Compute success rate and filter out unsuccessful attempts
    success_series = table_df.groupby(grouping).success.mean()
    table_df = table_df[table_df.success == 1]
    # Compute average metrics
    table_df = table_df.groupby(grouping)[['k', 'rate', 'time']].mean()
    table_df[['k', 'rate', 'time']] = table_df[['k', 'rate', 'time']].map(lambda x: f"{x:.2f}")
    # Add success rate to the table
    success_series = success_series.apply(lambda x: f"{x:.2f}")
    table_df = table_df.join(success_series).rename(
        {'success': 'success rate', 'rate': 'convergence rate', 'k': 'iterations'}, axis=1).unstack()
    text = table_df.to_latex(
        multicolumn_format='c',
        column_format = 'r' if exact else 'rr' + 'c'*table_df.shape[1],
        caption=f"Results for {method} method applied to {fname} {differentiation_text}, metrics are average metrics for succesful attempts.",
        label=f"tab:{method}_{fname}_{'exact' if exact else 'fd'}".replace(' ', '_'))
    text = text[:13] + "\n\\centering" + text[13:]
    return table_df, text

SyntaxError: f-string: expecting '}' (37281768.py, line 41)

In [6]:
functions = ['extended_rosenbrock', 'generalized_broyden', 'banded_trigonometric']
fnames = ['Extended Rosenbrock', 'Generalized Broyden', 'Banded Trigonometric']
methods = ['Modified Newton', 'Truncated Newton']
exacts = [True, False]
hs = [10**x for x in range(-2, -14, -2)]
relatives = [True, False]
i = 0

for func, fname in zip(functions, fnames):
    try:
        df = load_results(func)
    except:
        print(f"Missing function {fname}")
        continue
    for method in methods:
        table_df, text = avg_table2(df, method, True, fname)
        print(text)
        print()
        i += 1
    for method in methods:
        for relative in [False, True]:
            table_df, text = avg_table2(df, method, False, fname, relative)
            print(text)
            print()
            i += 1

\begin{table}
\centering
\caption{Results for Modified Newton method applied to Extended Rosenbrock with exact gradient and hessian, metrics are average metrics for succesful attempts.}
\label{tab:Modified_Newton_Extended_Rosenbrock_exact}
\begin{tabular}{r}
\toprule
 & \multicolumn{2}{c}{iterations} & \multicolumn{2}{c}{convergence rate} & \multicolumn{2}{c}{time} & \multicolumn{2}{c}{success rate} \\
preconditioning & False & True & False & True & False & True & False & True \\
dimension &  &  &  &  &  &  &  &  \\
\midrule
3 & 31.91 & 28.91 & 1.99 & 1.51 & 0.03 & 0.02 & 1.00 & 1.00 \\
4 & 32.36 & 29.18 & 2.10 & 2.04 & 0.11 & 0.08 & 1.00 & 1.00 \\
5 & 32.64 & 30.09 & 2.09 & 2.12 & 1.04 & 0.65 & 1.00 & 1.00 \\
\bottomrule
\end{tabular}
\end{table}


\begin{table}
\centering
\caption{Results for Truncated Newton method applied to Extended Rosenbrock with exact gradient and hessian, metrics are average metrics for succesful attempts.}
\label{tab:Truncated_Newton_Extended_Rosenbrock_exact

AttributeError: 'DataFrame' object has no attribute 'relative'